<a href="https://colab.research.google.com/github/RajeshworM/Yield_Modelling_Automation/blob/main/fasal2_4Sept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
import numpy as np
# Step 1: Load & prepare data
import pandas as pd

# Upload CSV file in Colab
from google.colab import files
uploaded = files.upload()

# Read data
df = pd.read_csv(list(uploaded.keys())[0])
# --- Step 0.1: Load data ---
df = pd.read_csv("data.csv")

# --- Step 0.2: Filter years 2005–2025 ---
df = df[(df['year'] >= 2005) & (df['year'] <= 2025)]

# --- Step 0.3: Drop rows with missing yield ---
df = df.dropna(subset=['yield'])

# --- Step 0.4: Define variable groups ---

# Economic
econ_vars = [
    'econ_inc_rev1', 'econ_inc_rev2', 'econ_inc_rev3',
    'econ_inc_rev4', 'econ_inc_rev5', 'econ_inc_rev6'
]

# Irrigation
irrigation_vars = [
    'gw_reciprocal', 'reservoir_intensity', 'canal_intensity',
    'well_intensity', 'tank_intensity'
]

# Stress
stress_vars = ['heat_stress', 'drought']

# Rainfall (weekly)
rainfall_weeks = [f'rf_week{i}' for i in range(20, 43)]  # 20–42 inclusive

# Temperature (weekly: max & min)
tmax_weeks = [f'tmax_week{i}' for i in range(20, 43)]
tmin_weeks = [f'tmin_week{i}' for i in range(20, 43)]

# Combine all predictors
all_predictors = econ_vars + irrigation_vars + stress_vars + rainfall_weeks + tmax_weeks + tmin_weeks

# --- Step 0.5: Impute missing values district-wise ---
# Forward fill + backward fill within each district
df[all_predictors] = (
    df.groupby('district_id')[all_predictors]
      .transform(lambda g: g.ffill().bfill())
)

# Fill any leftovers with median (across dataset)
for col in all_predictors:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())


# --- Step 0.6: Drop districts with <2 time points ---
district_counts = df.groupby('district_id')['year'].nunique()
valid_districts = district_counts[district_counts >= 2].index
df = df[df['district_id'].isin(valid_districts)]

print("Final shape:", df.shape)
print("Years span:", df['year'].min(), "to", df['year'].max())
print("Unique districts:", df['district_id'].nunique())
print("Example columns:", df.columns[:20].tolist())


Saving data.csv to data (11).csv
Final shape: (3087, 210)
Years span: 2005 to 2023
Unique districts: 166
Example columns: ['state', 'year', 'district', 'area', 'yield', 'nia', 'canal', 'well', 'tank', 'others', 'fert1', 'fert2', 'fert3', 'fert4', 'fert5', 'fert6', 'msp', 'price_crop1m11', 'price_crop2m11', 'price_crop3m11']


In [53]:
# Save cleaned dataset
df.to_csv('cleaned_agri_data.csv', index=False)
print("Cleaned data saved as 'cleaned_agri_data.csv'")
from google.colab import files

files.download('cleaned_agri_data.csv')


Cleaned data saved as 'cleaned_agri_data.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
import statsmodels.formula.api as smf

# Rename dependent variable
df = df.rename(columns={'yield':'yield_val'})
dep_var = 'yield_val'

# Auto-detect economic variables
econ_vars = [c for c in df.columns if 'econ' in c.lower()]
print("Detected economic variables:", econ_vars)

# Filter cluster 6
cluster_data = df[df['zone_cluster'] == 6].copy()

selected = []
current_avg_var = None

def run_fe(data, y, x):
    formula = f"{y} ~ {x} + C(district_id) + C(year)"
    model = smf.ols(formula, data=data).fit(cov_type="HC3")
    coef = model.params.get(x, None)
    pval = model.pvalues.get(x, None)
    return coef, pval

print(f"\n--- Processing zone_cluster=6 ---")

for var in econ_vars:
    if var not in cluster_data.columns:
        continue
    coef, pval = run_fe(cluster_data, dep_var, var)
    if coef is not None:
        print(f"Variable: {var}, coef={coef:.4f}, pval={pval:.4f}")
        if coef > 0 and pval < 0.10:
            if not selected:
                selected = [var]
                current_avg_var = var
                print(f"   --> Keeping {var} as first candidate")
            else:
                # Average with current selection
                cluster_data['econ_avg'] = cluster_data[selected + [var]].mean(axis=1)
                coef_avg, pval_avg = run_fe(cluster_data, dep_var, 'econ_avg')
                if coef_avg > 0 and pval_avg < 0.10 and pval_avg < pval:
                    selected.append(var)
                    current_avg_var = 'econ_avg'
                    print(f"   --> Improved with average: {selected}")
                else:
                    print(f"   --> Dropping {var}, no improvement")

print("\nFinal selected econ variable(s) for cluster 6:", selected)
# Create the averaged economic variable
cluster_data['final_econ'] = cluster_data[selected].mean(axis=1)

# Run final FE regression
formula = 'yield_val ~ final_econ'
final_model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")

# Display summary
print(final_model.summary())
# Create averaged economic variable
cluster_data['final_econ'] = cluster_data[selected].mean(axis=1)

# Run regression without district/year fixed effects
import statsmodels.formula.api as smf

test_vars = ['final_econ', 'gw_reciprocal']
formula = 'yield_val ~ ' + ' + '.join(test_vars)
model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")  # robust SE


# Variables for the current model
test_vars = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']

# Create formula
formula = 'yield_val ~ ' + ' + '.join(test_vars)

# Run OLS regression with robust SE
model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")

# Show summary
print(model.summary())


Detected economic variables: ['econ_incentive', 'econ_inc_rev1', 'econ_inc_rev2', 'econ_inc_rev3', 'econ_inc_rev4', 'econ_inc_rev5', 'econ_inc_rev6']

--- Processing zone_cluster=6 ---
Variable: econ_incentive, coef=-5.1062, pval=0.7241
Variable: econ_inc_rev1, coef=0.0107, pval=0.0001
   --> Keeping econ_inc_rev1 as first candidate
Variable: econ_inc_rev2, coef=0.0092, pval=0.0001
   --> Improved with average: ['econ_inc_rev1', 'econ_inc_rev2']
Variable: econ_inc_rev3, coef=0.0110, pval=0.0000
   --> Dropping econ_inc_rev3, no improvement
Variable: econ_inc_rev4, coef=0.0092, pval=0.0003
   --> Improved with average: ['econ_inc_rev1', 'econ_inc_rev2', 'econ_inc_rev4']
Variable: econ_inc_rev5, coef=0.0100, pval=0.0002
   --> Improved with average: ['econ_inc_rev1', 'econ_inc_rev2', 'econ_inc_rev4', 'econ_inc_rev5']
Variable: econ_inc_rev6, coef=0.0095, pval=0.0002
   --> Improved with average: ['econ_inc_rev1', 'econ_inc_rev2', 'econ_inc_rev4', 'econ_inc_rev5', 'econ_inc_rev6']

Final 

***step 2***

In [58]:
# Define rainfall stages and weeks
rain_stages = {
    'planting': range(20, 29),
    'vegetative': range(29, 34),
    'reproductive': range(34, 39),
    'ripening': range(39, 43)
}

# Start with previously selected variables
current_vars = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']

# Dictionary to store positive and negative groups
rain_groups = {}

# Function to run OLS
def run_ols(data, y, x_vars):
    formula = f"{y} ~ {' + '.join(x_vars)}"
    model = smf.ols(formula, data=data).fit(cov_type="HC3")
    return model

# Loop through stages
for stage, weeks in rain_stages.items():
    pos_group = []
    neg_group = []
    rain_vars = [f'rf_week{i}' for i in weeks if f'rf_week{i}' in cluster_data.columns]

    for var in rain_vars:
        model = run_ols(cluster_data, 'yield_val', current_vars + [var])
        coef = model.params[var]
        pval = model.pvalues[var]
        print(f"{stage} - {var}: coef={coef:.4f}, pval={pval:.4f}")

        if pval < 0.10:
            if coef > 0:
                pos_group.append(var)
                print(f"   --> Added to positive group")
            elif coef < 0:
                neg_group.append(var)
                print(f"   --> Added to negative group")
        else:
            print(f"   --> Not significant, dropped")

    # Create average columns for the stage
    if pos_group:
        cluster_data[f'{stage}_rain_pos'] = cluster_data[pos_group].mean(axis=1)
    if neg_group:
        cluster_data[f'{stage}_rain_neg'] = cluster_data[neg_group].mean(axis=1)

    rain_groups[stage] = {'positive': pos_group, 'negative': neg_group}

print("\nAll rainfall groups:")
for stage, groups in rain_groups.items():
    print(f"{stage}: Positive: {groups['positive']}, Negative: {groups['negative']}")

# Initialize regression variables
regression_vars = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']

# Add rainfall averages if they exist
for stage in ['planting', 'reproductive', 'ripening']:
    for grp in ['pos', 'neg']:
        col = f"{stage}_rain_{grp}"
        if col in cluster_data.columns:
            regression_vars.append(col)

# Run OLS regression
formula = 'yield_val ~ ' + ' + '.join(regression_vars)
model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")

# Show summary
print(model.summary())



planting - rf_week20: coef=-2.8513, pval=0.0783
   --> Added to negative group
planting - rf_week21: coef=-0.4944, pval=0.7710
   --> Not significant, dropped
planting - rf_week22: coef=3.1924, pval=0.0022
   --> Added to positive group
planting - rf_week23: coef=-1.7726, pval=0.0031
   --> Added to negative group
planting - rf_week24: coef=1.8302, pval=0.0024
   --> Added to positive group
planting - rf_week25: coef=0.1297, pval=0.8865
   --> Not significant, dropped
planting - rf_week26: coef=-0.7095, pval=0.1801
   --> Not significant, dropped
planting - rf_week27: coef=0.9189, pval=0.0220
   --> Added to positive group
planting - rf_week28: coef=0.7477, pval=0.0693
   --> Added to positive group
vegetative - rf_week29: coef=0.4049, pval=0.5001
   --> Not significant, dropped
vegetative - rf_week30: coef=0.4076, pval=0.3477
   --> Not significant, dropped
vegetative - rf_week31: coef=-0.1201, pval=0.8050
   --> Not significant, dropped
vegetative - rf_week32: coef=-0.1128, pval=0.73

                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.575
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     159.3
Date:                Thu, 04 Sep 2025   Prob (F-statistic):          9.55e-170
Time:                        08:42:29   Log-Likelihood:                -6054.1
No. Observations:                 786   AIC:                         1.213e+04
Df Residuals:                     776   BIC:                         1.217e+04
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1641.22

***Step 3***

In [59]:
# Temperature stages and weeks
temp_stages = {
    'planting': range(20, 29),
    'vegetative': range(29, 34),
    'reproductive': range(34, 39),
    'ripening': range(39, 43)
}

# Temperature types
temp_types = ['tmax', 'tmin']

# Loop through temperature types and stages
for temp in temp_types:
    for stage, weeks in temp_stages.items():
        pos_group = []
        neg_group = []

        temp_vars = [f'{temp}_week{i}' for i in weeks if f'{temp}_week{i}' in cluster_data.columns]

        # Current model variables include economy, irrigation, and **selected rainfall averages**
        current_vars_temp = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']
        for r_stage in ['planting', 'reproductive', 'ripening']:
            for grp in ['pos', 'neg']:
                col = f"{r_stage}_rain_{grp}"
                if col in cluster_data.columns:
                    current_vars_temp.append(col)

        # Loop through weekly temp variables
        for var in temp_vars:
            model = smf.ols(f"yield_val ~ {' + '.join(current_vars_temp + [var])}", data=cluster_data).fit(cov_type="HC3")
            coef = model.params[var]
            pval = model.pvalues[var]
            print(f"{temp} - {stage} - {var}: coef={coef:.4f}, pval={pval:.4f}")

            if pval < 0.10:
                if coef > 0:
                    pos_group.append(var)
                    print(f"   --> Added to positive group")
                elif coef < 0:
                    neg_group.append(var)
                    print(f"   --> Added to negative group")
            else:
                print(f"   --> Not significant, dropped")

        # Create averaged columns for stage
        if pos_group:
            cluster_data[f'{stage}_{temp}_pos'] = cluster_data[pos_group].mean(axis=1)
        if neg_group:
            cluster_data[f'{stage}_{temp}_neg'] = cluster_data[neg_group].mean(axis=1)

# Start with economy and irrigation
final_vars = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']

# Add selected rainfall averages
for r_stage in ['planting', 'reproductive', 'ripening']:
    for grp in ['pos', 'neg']:
        col = f"{r_stage}_rain_{grp}"
        if col in cluster_data.columns:
            final_vars.append(col)

# Add selected temperature averages
for temp in ['tmax', 'tmin']:
    for stage in ['planting', 'vegetative', 'reproductive', 'ripening']:
        for grp in ['pos', 'neg']:
            col = f"{stage}_{temp}_{grp}"
            if col in cluster_data.columns:
                final_vars.append(col)

# Run final OLS regression
formula = 'yield_val ~ ' + ' + '.join(final_vars)
final_model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")

# Display summary
print(final_model.summary())



tmax - planting - tmax_week20: coef=-46.2550, pval=0.0000
   --> Added to negative group
tmax - planting - tmax_week21: coef=-32.7348, pval=0.0000
   --> Added to negative group
tmax - planting - tmax_week22: coef=-15.2421, pval=0.0234
   --> Added to negative group
tmax - planting - tmax_week23: coef=0.8321, pval=0.9281
   --> Not significant, dropped
tmax - planting - tmax_week24: coef=-18.4750, pval=0.0603
   --> Added to negative group
tmax - planting - tmax_week25: coef=6.2637, pval=0.4719
   --> Not significant, dropped
tmax - planting - tmax_week26: coef=16.0575, pval=0.0982
   --> Added to positive group
tmax - planting - tmax_week27: coef=-0.9515, pval=0.9241
   --> Not significant, dropped
tmax - planting - tmax_week28: coef=-15.8099, pval=0.1977
   --> Not significant, dropped
tmax - vegetative - tmax_week29: coef=19.6613, pval=0.1718
   --> Not significant, dropped
tmax - vegetative - tmax_week30: coef=30.4311, pval=0.0368
   --> Added to positive group
tmax - vegetative - 

**Step 4**

In [60]:
# Irrigation variables to test
irrigation_vars = ['canal_intensity', 'well_intensity', 'tank_intensity', 'oth_intensity']

# Initialize groups
irrigation_pos = []
irrigation_neg = []

# Current model variables (all previously selected)
current_vars = ['final_econ', 'gw_reciprocal', 'reservoir_intensity']

# Add rainfall averages
for r_stage in ['planting', 'reproductive', 'ripening']:
    for grp in ['pos', 'neg']:
        col = f"{r_stage}_rain_{grp}"
        if col in cluster_data.columns:
            current_vars.append(col)

# Add temperature averages
for temp in ['tmax', 'tmin']:
    for stage in ['planting', 'vegetative', 'reproductive', 'ripening']:
        for grp in ['pos', 'neg']:
            col = f"{stage}_{temp}_{grp}"
            if col in cluster_data.columns:
                current_vars.append(col)

# Loop through irrigation variables
for var in irrigation_vars:
    if var not in cluster_data.columns:
        continue
    model = smf.ols(f"yield_val ~ {' + '.join(current_vars + [var])}", data=cluster_data).fit(cov_type="HC3")
    coef = model.params[var]
    pval = model.pvalues[var]
    print(f"{var}: coef={coef:.4f}, pval={pval:.4f}")

    if pval < 0.10:
        if coef > 0:
            irrigation_pos.append(var)
            print(f"   --> Added to positive group")
        elif coef < 0:
            irrigation_neg.append(var)
            print(f"   --> Added to negative group")
    else:
        print(f"   --> Not significant, dropped")

# Create averaged columns for irrigation
if irrigation_pos:
    cluster_data['irrigation_pos'] = cluster_data[irrigation_pos].mean(axis=1)
if irrigation_neg:
    cluster_data['irrigation_neg'] = cluster_data[irrigation_neg].mean(axis=1)

print("\nFinal irrigation groups:")
print("Positive:", irrigation_pos)
print("Negative:", irrigation_neg)

# Start with economy
final_vars = ['final_econ']

# Add rainfall averages
for r_stage in ['planting', 'reproductive', 'ripening']:
    for grp in ['pos', 'neg']:
        col = f"{r_stage}_rain_{grp}"
        if col in cluster_data.columns:
            final_vars.append(col)

# Add temperature averages
for temp in ['tmax', 'tmin']:
    for stage in ['planting', 'vegetative', 'reproductive', 'ripening']:
        for grp in ['pos', 'neg']:
            col = f"{stage}_{temp}_{grp}"
            if col in cluster_data.columns:
                final_vars.append(col)

# Add irrigation averages if they exist
for grp in ['pos', 'neg']:
    col = f"irrigation_{grp}"
    if col in cluster_data.columns:
        final_vars.append(col)

# Run final OLS regression
formula = 'yield_val ~ ' + ' + '.join(final_vars)
full_model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")

# Display summary
print(full_model.summary())


canal_intensity: coef=-50.0788, pval=0.0139
   --> Added to negative group
well_intensity: coef=-1.0920, pval=0.8432
   --> Not significant, dropped
tank_intensity: coef=66252.3842, pval=0.0258
   --> Added to positive group
oth_intensity: coef=988.2181, pval=0.2784
   --> Not significant, dropped

Final irrigation groups:
Positive: ['tank_intensity']
Negative: ['canal_intensity']
                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     27.28
Date:                Thu, 04 Sep 2025   Prob (F-statistic):           2.85e-70
Time:                        10:54:49   Log-Likelihood:                -5921.3
No. Observations:                 786   AIC:                         1.188e+04
Df Residuals:                     766   BIC:                   

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 19, but rank is 18
  warnings.warn('covariance of constraints does not have full '


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Select the numeric predictors for VIF calculation
X = cluster_data[final_vars]  # all predictors used in the final regression
X = X.dropna()  # ensure no missing values

# Add constant for VIF calculation
X_const = sm.add_constant(X)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print(vif_data.sort_values("VIF", ascending=False))


                  feature          VIF
0                   const  1103.187935
15    vegetative_tmin_neg    20.003893
14      planting_tmin_neg    18.398573
16  reproductive_tmin_neg    11.181062
17      ripening_tmin_neg     5.874142
9       planting_tmax_neg     5.772266
10    vegetative_tmax_pos     5.311076
13      ripening_tmax_neg     5.147016
12  reproductive_tmax_neg     5.131309
11    vegetative_tmax_neg     4.494820
8       planting_tmax_pos     1.965100
3       planting_rain_neg     1.802358
2       planting_rain_pos     1.749979
6       ripening_rain_pos     1.706795
5   reproductive_rain_neg     1.677967
4   reproductive_rain_pos     1.592382
1              final_econ     1.588937
7       ripening_rain_neg     1.375293
19         irrigation_neg     1.373225
18         irrigation_pos     1.059846


In [ ]:
# 1. Prepare final_vars dynamically but only include existing columns
final_vars = []

# Economy
if 'final_econ' in cluster_data.columns:
    final_vars.append('final_econ')

# Rainfall averages
for r_stage in ['planting', 'reproductive', 'ripening']:
    for grp in ['pos', 'neg']:
        col = f"{r_stage}_rain_{grp}"
        if col in cluster_data.columns:
            final_vars.append(col)

# Temperature averages
for temp in ['tmax', 'tmin']:
    for stage in ['planting', 'vegetative', 'reproductive', 'ripening']:
        for grp in ['pos', 'neg']:
            col = f"{stage}_{temp}_{grp}"
            if col in cluster_data.columns:
                final_vars.append(col)

# Irrigation averages
for grp in ['pos', 'neg']:
    col = f"irrigation_{grp}"
    if col in cluster_data.columns:
        final_vars.append(col)

print("Final variables actually found in data:")
print(final_vars)

# 2. Run final regression with only existing columns
formula = 'yield_val ~ ' + ' + '.join(final_vars)
full_model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")
print(full_model.summary())


Final variables actually found in data:
['final_econ', 'planting_rain_pos', 'planting_rain_neg', 'reproductive_rain_pos', 'reproductive_rain_neg', 'ripening_rain_pos', 'ripening_rain_neg', 'planting_tmax_pos', 'planting_tmax_neg', 'vegetative_tmax_pos', 'vegetative_tmax_neg', 'reproductive_tmax_neg', 'ripening_tmax_neg', 'ripening_tmin_neg', 'irrigation_pos', 'irrigation_neg']
                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     20.64
Date:                Thu, 04 Sep 2025   Prob (F-statistic):           3.53e-47
Time:                        09:04:59   Log-Likelihood:                -5960.1
No. Observations:                 786   AIC:                         1.195e+04
Df Residuals:                     769   BIC:                       

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 16, but rank is 15
  warnings.warn('covariance of constraints does not have full '


**Step 5**

In [61]:
# -------------------------------
# 1. Run initial regression with all final_vars
# -------------------------------
formula = 'yield_val ~ ' + ' + '.join(final_vars)
model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")
print("Initial regression summary:")
print(model.summary())

# -------------------------------
# 2. Drop insignificant variables (p > 0.10)
# -------------------------------
insig_vars = [var for var, pval in model.pvalues.items() if pval > 0.10 and var != 'Intercept']
print(f"Insignificant variables to drop: {insig_vars}")

for var in insig_vars:
    final_vars.remove(var)

# -------------------------------
# 3. Re-run regression after dropping insignificant variables
# -------------------------------
model = smf.ols('yield_val ~ ' + ' + '.join(final_vars), data=cluster_data).fit(cov_type="HC3")
print("\nRegression after dropping insignificant variables:")
print(model.summary())

# -------------------------------
# 4. VIF calculation and multicollinearity correction
# -------------------------------
X = cluster_data[final_vars].dropna()
X_const = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data['feature'] = X_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print("\nVIF before correction:")
print(vif_data.sort_values('VIF', ascending=False))

# Iteratively drop variables with VIF > 10
high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()
while high_vif:
    high_vif_vars = [var for var in high_vif if var != 'const']
    if not high_vif_vars:
        break
    drop_var = high_vif_vars[0]
    print(f"Dropping {drop_var} due to high VIF")
    final_vars.remove(drop_var)

    X = cluster_data[final_vars].dropna()
    X_const = sm.add_constant(X)
    vif_data = pd.DataFrame()
    vif_data['feature'] = X_const.columns
    vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
    high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()

# -------------------------------
# 5. Final regression after dropping insignificant and high-VIF variables
# -------------------------------
final_model = smf.ols('yield_val ~ ' + ' + '.join(final_vars), data=cluster_data).fit(cov_type="HC3")
print("\nFinal regression after dropping insignificant variables and correcting VIF:")
print(final_model.summary())
print("\nFinal VIF values:")
print(vif_data.sort_values('VIF', ascending=False))


Initial regression summary:
                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     27.28
Date:                Thu, 04 Sep 2025   Prob (F-statistic):           2.85e-70
Time:                        10:56:59   Log-Likelihood:                -5921.3
No. Observations:                 786   AIC:                         1.188e+04
Df Residuals:                     766   BIC:                         1.198e+04
Df Model:                          19                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
In

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 19, but rank is 18
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 13
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     168.7
Date:                Thu, 04 Sep 2025   Prob (F-statistic):          4.00e-215
Time:                        10:56:59   Log-Likelihood:                -5928.5
No. Observations:                 786   AIC:                         1.189e+04
Df Residuals:                     771   BIC:                         1.196e+04
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2997.33

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 12
  warnings.warn('covariance of constraints does not have full '


**Step 6**

In [63]:
# Add heat_stress and drought if they exist
for var in ['heat_stress', 'drought']:
    if var in cluster_data.columns:
        final_vars.append(var)

print("Final variables including heat_stress and drought:")
print(final_vars)

# -------------------------------
# 1. Initial regression
# -------------------------------
formula = 'yield_val ~ ' + ' + '.join(final_vars)
model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")
print("Initial regression summary:")
print(model.summary())

# -------------------------------
# 2. Drop insignificant variables (p > 0.10)
# -------------------------------
insig_vars = [var for var, pval in model.pvalues.items() if pval > 0.10 and var != 'Intercept']
print(f"Insignificant variables to drop: {insig_vars}")

for var in insig_vars:
    final_vars.remove(var)

# -------------------------------
# 3. Re-run regression after dropping insignificant variables
# -------------------------------
model = smf.ols('yield_val ~ ' + ' + '.join(final_vars), data=cluster_data).fit(cov_type="HC3")
print("\nRegression after dropping insignificant variables:")
print(model.summary())

# -------------------------------
# 4. VIF calculation and multicollinearity correction
# -------------------------------
X = cluster_data[final_vars].dropna()
X_const = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data['feature'] = X_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print("\nVIF before correction:")
print(vif_data.sort_values('VIF', ascending=False))

# Iteratively drop variables with VIF > 10
high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()
while high_vif:
    high_vif_vars = [var for var in high_vif if var != 'const']
    if not high_vif_vars:
        break
    drop_var = high_vif_vars[0]
    print(f"Dropping {drop_var} due to high VIF")
    final_vars.remove(drop_var)

    X = cluster_data[final_vars].dropna()
    X_const = sm.add_constant(X)
    vif_data = pd.DataFrame()
    vif_data['feature'] = X_const.columns
    vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
    high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()

# -------------------------------
# 5. Final regression after dropping insignificant and high-VIF variables
# -------------------------------
final_model = smf.ols('yield_val ~ ' + ' + '.join(final_vars), data=cluster_data).fit(cov_type="HC3")
print("\nFinal regression after including heat_stress and drought, dropping insignificant variables, and correcting VIF:")
print(final_model.summary())
print("\nFinal VIF values:")
print(vif_data.sort_values('VIF', ascending=False))


Final variables including heat_stress and drought:
['final_econ', 'planting_rain_neg', 'reproductive_rain_neg', 'ripening_rain_neg', 'planting_tmax_pos', 'planting_tmax_neg', 'vegetative_tmax_pos', 'ripening_tmax_neg', 'ripening_tmin_neg', 'irrigation_pos', 'irrigation_neg', 'heat_stress', 'drought']
Initial regression summary:
                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     26.17
Date:                Thu, 04 Sep 2025   Prob (F-statistic):           9.55e-50
Time:                        10:58:21   Log-Likelihood:                -5970.1
No. Observations:                 786   AIC:                         1.197e+04
Df Residuals:                     772   BIC:                         1.203e+04
Df Model:                          13 

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 13, but rank is 12
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 12, but rank is 11
  warnings.warn('covariance of constraints does not have full '


In [65]:
# Start with final_vars from previous steps (after significance + VIF correction)
current_vars = final_vars.copy()

# Interaction candidates
interaction_candidates = []

# Rain × Irrigation
rain_vars = [v for v in current_vars if '_rain_' in v]
irrig_vars = [v for v in current_vars if 'irrigation_' in v]
for r in rain_vars:
    for i in irrig_vars:
        interaction_candidates.append((r, i))

# Rain × Reservoir
if 'reservoir_intensity' in cluster_data.columns:
    for r in rain_vars:
        interaction_candidates.append((r, 'reservoir_intensity'))

# Irrigation × Reservoir
if 'reservoir_intensity' in cluster_data.columns:
    for i in irrig_vars:
        interaction_candidates.append((i, 'reservoir_intensity'))

# Quadratic terms for rainfall
quadratic_candidates = [(r,) for r in rain_vars]

# Function to test a single term
def test_term(term_name, term_values):
    cluster_data[term_name] = term_values
    formula = 'yield_val ~ ' + ' + '.join(current_vars + [term_name])
    model = smf.ols(formula, data=cluster_data).fit(cov_type="HC3")
    pval = model.pvalues[term_name]
    if pval <= 0.10:
        print(f"Keeping {term_name}, p-value={pval:.4f}")
        current_vars.append(term_name)
    else:
        print(f"Dropping {term_name}, p-value={pval:.4f}")
        cluster_data.drop(columns=[term_name], inplace=True)

# -------------------------------
# Test interactions one by one
# -------------------------------
for pair in interaction_candidates:
    term_name = f"{pair[0]}_x_{pair[1]}"
    term_values = cluster_data[pair[0]] * cluster_data[pair[1]]
    test_term(term_name, term_values)

# -------------------------------
# Test quadratic terms one by one
# -------------------------------
for quad in quadratic_candidates:
    term_name = f"{quad[0]}_sq"
    term_values = cluster_data[quad[0]] ** 2
    test_term(term_name, term_values)

# -------------------------------
# After testing all, run final regression
# -------------------------------
final_model = smf.ols('yield_val ~ ' + ' + '.join(current_vars), data=cluster_data).fit(cov_type="HC3")
print(final_model.summary())

# -------------------------------
# Check VIF and remove high collinearity variables
# -------------------------------
X = cluster_data[current_vars].dropna()
X_const = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data['feature'] = X_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print(vif_data.sort_values('VIF', ascending=False))


Dropping planting_rain_neg_x_irrigation_pos, p-value=0.9323
Keeping planting_rain_neg_x_irrigation_neg, p-value=0.0631
Dropping reproductive_rain_neg_x_irrigation_pos, p-value=0.3685
Dropping reproductive_rain_neg_x_irrigation_neg, p-value=0.8198
Dropping ripening_rain_neg_x_irrigation_pos, p-value=0.4264
Dropping ripening_rain_neg_x_irrigation_neg, p-value=0.5861
Keeping planting_rain_neg_x_reservoir_intensity, p-value=0.0005
Dropping reproductive_rain_neg_x_reservoir_intensity, p-value=0.6397
Dropping ripening_rain_neg_x_reservoir_intensity, p-value=0.2157
Dropping irrigation_pos_x_reservoir_intensity, p-value=0.3036
Dropping irrigation_neg_x_reservoir_intensity, p-value=0.1847
Dropping planting_rain_neg_sq, p-value=0.9596
Dropping reproductive_rain_neg_sq, p-value=0.4039
Dropping ripening_rain_neg_sq, p-value=0.8104
                            OLS Regression Results                            
Dep. Variable:              yield_val   R-squared:                       0.664
Model:     

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 14, but rank is 13
  warnings.warn('covariance of constraints does not have full '


In [66]:
!pip install linearmodels
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.api as sm
from scipy import stats
import numpy as np

# Ensure panel format
cluster_panel = cluster_data.set_index(['district_id', 'year'])

# Independent variables (final selected variables from OLS)
exog_vars = current_vars.copy()  # final selected variables from OLS/VIF/significance
exog = sm.add_constant(cluster_panel[exog_vars])

# Dependent variable
y = cluster_panel['yield_val']

# Fixed Effects
fe_model = PanelOLS(y, cluster_panel[exog_vars], entity_effects=True).fit(cov_type='robust')
print("Fixed Effects Model Summary:")
print(fe_model.summary)

# Random Effects
re_model = RandomEffects(y, exog).fit(cov_type='robust')
print("\nRandom Effects Model Summary:")
print(re_model.summary)

# Hausman Test
def hausman(fe, re):
    b_fe = fe.params
    b_re = re.params
    v_fe = fe.cov
    v_re = re.cov

    diff = b_fe - b_re
    cov_diff = v_fe - v_re
    stat = np.dot(np.dot(diff.T, np.linalg.inv(cov_diff)), diff)
    df = diff.shape[0]
    pval = 1 - stats.chi2.cdf(stat, df)
    return stat, pval

stat, pval = hausman(fe_model, re_model)
print(f"\nHausman Test: Chi2={stat:.4f}, p-value={pval:.4f}")
if pval < 0.05:
    print("Reject H0 → Fixed Effects preferred")
else:
    print("Fail to reject H0 → Random Effects preferred")


Fixed Effects Model Summary:
                          PanelOLS Estimation Summary                           
Dep. Variable:              yield_val   R-squared:                        0.2484
Estimator:                   PanelOLS   R-squared (Between):              0.0057
No. Observations:                 786   R-squared (Within):               0.2484
Date:                Thu, Sep 04 2025   R-squared (Overall):              0.0098
Time:                        10:59:43   Log-likelihood                   -5663.5
Cov. Estimator:                Robust                                           
                                        F-statistic:                      17.213
Entities:                          43   P-value                           0.0000
Avg Obs:                       18.279   Distribution:                  F(14,729)
Min Obs:                       7.0000                                           
Max Obs:                       19.000   F-statistic (robust):             22.581